In [ ]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 16.1 MB/s eta 0:00:00


In [ ]:
!pip install gdown

In [ ]:
import stanza
stanza.download('ru')
nlp = stanza.Pipeline("ru", package="Taiga")
from stanza.utils.conll import CoNLL
import json
import os
import re
import nltk
from string import punctuation
import gdown
import zipfile

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package        |
------------------------------
| tokenize  | taiga          |
| pos       | taiga_charlm   |
| lemma     | taiga_nocharlm |
| depparse  | taiga_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted b

In [ ]:
os.mkdir("conllu")

In [ ]:
def txt_to_conllu(filename: str) -> None:
    """
    Создает файл .conllu с мофологической разметкой файлов .txt с текстами
    статей, предварительно обрабатывая их с помощью Stanza.

    Args:
        filename (str): Путь к входному .txt файлу с текстом.

    """
    with open(filename, 'r', encoding='UTF-8') as f:
        lines = f.readlines()

    # Извлeчение метаданных и объединение текстового содержимого
    metadata = json.loads(lines[0].strip())
    full_text = ''.join(lines[2:-1])

    # POS-tagging текста с помощью Stanza
    doc = nlp(full_text)

    # Формирование названия выходного файла
    c_filename = os.path.join('conllu', f"{metadata['link'].split('/')[-2]}.conllu")

    # Запись метаданных и CoNLL данных в файл
    with open(c_filename, "w", encoding="UTF-8") as f:
        f.write(f"# metadata = автор: {metadata['author']}, заголовок: {metadata['title']}, ссылка: {metadata['link']}\n")

    # Сохранение данных в формате CoNLL
    CoNLL.write_doc2conll(doc, c_filename, mode="a")


In [ ]:
url = f'https://drive.google.com/uc?id={"1DzJYVeDNautgBNv6eebNf6-XUoryIjaF"}'
output = 'corpus.zip'

# Скачивание zip-файла с текстами статей и метаданными
gdown.download(url, output)

# Распаковка zip-файла
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('corpus')

# Удаление zip-файла после распаковки
os.remove(output)

Downloading...
From: https://drive.google.com/uc?id=1DzJYVeDNautgBNv6eebNf6-XUoryIjaF
To: /content/corpus.zip
100%|██████████| 3.94M/3.94M [00:00<00:00, 110MB/s]


In [ ]:
directory = "/content/corpus/corpus"
for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    if os.path.isfile(file_path):
        txt_to_conllu(file_path)
